In [15]:
import pandas as pd
revenue_df = pd.read_csv('NetflixsRevenue2018toQ2_2020.csv')

In [16]:
monthly_revenue_df = revenue_df.copy()

In [17]:
quarter_columns = monthly_revenue_df.columns[1:]  # All columns except 'Area'
for column in quarter_columns:
    monthly_revenue_df[column] = monthly_revenue_df[column] / 3

In [18]:
monthly_revenue_melted = pd.melt(monthly_revenue_df, id_vars=['Area'], var_name='Quarter', value_name='Monthly_Revenue')

In [19]:
# Add a date column by inferring from the 'Quarter' column
def convert_to_month(quarter_str):
    quarter_mapping = {
        'Q1': '01', 'Q2': '04', 'Q3': '07', 'Q4': '10'
    }
    year = quarter_str.split(' - ')[1]
    quarter = quarter_str.split(' - ')[0]
    month = quarter_mapping[quarter]
    return f'{year}-{month}'

In [20]:
monthly_revenue_melted['Date'] = monthly_revenue_melted['Quarter'].apply(convert_to_month)
monthly_revenue_melted['Date'] = pd.to_datetime(monthly_revenue_melted['Date'])

In [21]:
monthly_revenue_melted = monthly_revenue_melted.drop(columns=['Quarter'])

In [22]:
monthly_revenue_melted = monthly_revenue_melted.sort_values(by='Date')

In [23]:
print(monthly_revenue_melted.head())

                              Area  Monthly_Revenue       Date
0         United States and Canada     6.587190e+08 2018-01-01
1  Europe, Middle East, and Africa     2.955497e+08 2018-01-01
2                    Latin America     1.800607e+08 2018-01-01
3                     Asia-Pacific     6.637233e+07 2018-01-01
4         United States and Canada     6.831820e+08 2018-04-01


In [24]:
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import train_test_split

In [25]:
region = 'United States and Canada'
region_data = monthly_revenue_melted[monthly_revenue_melted['Area'] == region]

In [26]:
# Prepare the time series data and set frequency
time_series = region_data[['Date', 'Monthly_Revenue']].set_index('Date')
time_series = time_series.asfreq('M')  # Set frequency to monthly


C:\Users\arana\AppData\Local\Temp\ipykernel_13856\2232824104.py:3: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  time_series = time_series.asfreq('M')  # Set frequency to monthly


In [27]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(time_series, test_size=0.1, shuffle=False)


In [28]:
# Train the ARIMA model
model = ARIMA(train_data, order=(5, 1, 0))  # order=(p, d, q)
arima_model = model.fit()

C:\Users\arana\AppData\Roaming\Python\Python311\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
C:\Program Files\Lib\site-packages\numpy\core\fromnumeric.py:3787: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Program Files\Lib\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
C:\Program Files\Lib\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


LinAlgError: Schur decomposition solver error.

In [29]:
# Predict the future values (next 3 months)
forecast_steps = 3
forecast = arima_model.forecast(steps=forecast_steps)

In [30]:
# Display the forecasted revenue
print(f"Forecasted revenue for the next {forecast_steps} months: \n{forecast}")

Forecasted revenue for the next 3 months: 
2020-04-01    9.160880e+08
2020-07-01    9.917329e+08
2020-10-01    1.044846e+09
Freq: QS-OCT, Name: predicted_mean, dtype: float64
